## Notebook for making predictions with deleted constant y samples


In [49]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import numpy as np
from sklearn.ensemble import StackingRegressor
import lightgbm as lgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import BaggingRegressor


### Reading dataset stored in other notebooks

In [50]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [51]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

In [52]:
# remove special characters from column names
X_A_train.columns = X_A_train.columns.str.replace(':', '_')
X_B_train.columns = X_B_train.columns.str.replace(':', '_')
X_C_train.columns = X_C_train.columns.str.replace(':', '_')

X_A_test.columns = X_A_test.columns.str.replace(':', '_')
X_B_test.columns = X_B_test.columns.str.replace(':', '_')
X_C_test.columns = X_C_test.columns.str.replace(':', '_')

In [ ]:
X_train = pd.concat([X_A_train, X_B_train, X_C_train])
Y_train = pd.concat([Y_A_train, Y_B_train, Y_C_train])

X_test = pd.concat([X_A_test, X_B_test, X_C_test])

# Stack model

In [59]:
catModel = CatBoostRegressor()

nnModel = MLPRegressor()

bagModel = BaggingRegressor()

metaModel = LinearRegression()

In [60]:
# stacking classifier
stackingModel = StackingRegressor(
    estimators=[('cat', catModel), ('nn', nnModel), ('bag', bagModel)],
    final_estimator= metaModel
)


In [61]:
# fit stacking models
stackingModel.fit(X_train, Y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004377 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7221
[LightGBM] [Info] Number of data points in the train set: 23224, number of used features: 44
[LightGBM] [Info] Start training from score 925.459494
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004264 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7153
[LightGBM] [Info] Number of data points in the train set: 18579, number of used features: 44
[LightGBM] [Info] Start training from score 948.228123
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002562 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7190
[LightGBM] [Info] Number of data points in the train set: 18579, number of used features: 44
[LightGBM] [Info] Start

StackingRegressor(estimators=[('rfr', RandomForestRegressor()),
                              ('nn', MLPRegressor()),
                              ('gbm', LGBMRegressor())],
                  final_estimator=LinearRegression())

## Preds

In [64]:
# preds for stacking models
stacking_preds = stackingModel.predict(X_test)


In [ ]:
cat_preds = pd.read_csv("./ext_preds/bestpred.csv")

In [66]:
stacking_preds[stacking_preds < 0] = 0
pred_mean = np.mean([cat_preds, stacking_preds], axis=0)
pred_mean[pred_mean < 0] = 0


In [ ]:
pred_mean.to_csv("./ext_preds/newstack_0.csv", index=False)